<a href="https://colab.research.google.com/github/AngeMervaneJdev/ML_AI_1/blob/main/SimplConv1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#[Problème 1] Création d'une classe de couche convolutionnelle unidimensionnelle qui limite le nombre de canaux à un

In [29]:
import numpy as np;
from scipy.sparse import dia_matrix
from keras.datasets import mnist
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [30]:

class SimpleConv1d:
    def __init__(self, weight, baias):
        self.optimizer = None

        self.W = weight
        self.B = baias
        self.dW = None
        self.dB = None
        self.idx =None
        self.X = None
        pass

    def forward(self, X):
        self.a = X.copy()
        idx1 = np.arange(self.W.shape[0])
        idx2 = np.arange(self.W.shape[0]-1 ).reshape(-1, 1)
        self.idx = idx1 + idx2
        A = np.sum(X[self.idx] * self.W.T,axis=1) + self.B
        return A
    def backward(self, dX,dA):

        dB = np.sum(dA,axis=0)
        dW = np.sum(dA[:,np.newaxis] *dX[self.idx],axis=0)
        da = dA.reshape(-1,1)
        data= np.repeat(da,dX.shape[0],axis=1)
        offsets= np.arange(self.output(data))
        d = dia_matrix((data,offsets),shape=(w.shape[0],x.shape[0])).toarray()
        dx = np.sum(d * w[:,np.newaxis],axis=0)
        
        return dX
    def output(self,X):
        n_in = X.shape[1]
        p = 0
        f = self.W.shape[0]
        s = 1 
        n_out = ((n_in + 2*p - f)/s)+1
        return n_out

#[Problème 2] Calcul de la taille de sortie après convolution unidimensionnelle

In [31]:
def output(X,p,f,s):
    n_in = X.shape[0]
    n_out = ((n_in + 2*p - f)/s)+1
    return n_out

#[Problème 3] Expérience d'une couche convolutionnelle unidimensionnelle avec un petit réseau

In [32]:
x = np.array([1,2,3,4])
w = np.array([3, 5, 7])
b = np.array([1])
x.shape,w.shape,b.shape

((4,), (3,), (1,))

In [33]:
sc1 = SimpleConv1d(w,b)
sc1.forward(x)

array([35, 50])

In [34]:
delta_a = np.array([10, 20])
delta_a.shape
delta_b = np.array([30])
delta_w = np.array([50, 80, 110])
delta_x = np.array([30, 110, 170, 140])

In [35]:
sc1 = SimpleConv1d(delta_w,delta_b)
sc1.backward(delta_x,delta_a)

array([ 30, 110, 170, 140])

In [36]:
x = np.array([1, 2, 3, 4])
w = np.array([3, 5, 7])

a = np.empty((2, 3))

indexes0 = np.array([0, 1, 2])
indexes1 = np.array([1, 2, 3])

a[0] = x[indexes0]*w # x[indexes0]は([1, 2, 3])である
a[1] = x[indexes1]*w # x[indexes1]は([2, 3, 4])である

a = a.sum(axis=1)
a

array([34., 49.])

#[Problème 4] Création d'une classe de couche convolutionnelle unidimensionnelle qui ne limite pas le nombre de canaux

In [37]:
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) # 
w = np.ones((3, 2, 3)) # 
b = np.array([1, 2, 3])

In [38]:
def output2(X,p,f,s):
    n_in = X.shape[1]
    p = p #
    f = f #
    s = s #
    n_out = ((n_in + 2*p - f)/s)+1
    return n_out

In [39]:
a = np.array([[16, 22], [17, 23], [18, 24]])
o = int(output2(x,0,w.shape[-1],1))
print(o)

nin, i = x.shape
print(nin,i)
oc, c, f = w.shape
print(oc,c,f)

#c * f 
idx_arr = np.arange(c*f).reshape(c,f)
idx_arr

2
2 4
3 2 3


array([[0, 1, 2],
       [3, 4, 5]])

In [40]:
idx1 = np.arange(w.shape[-1])

idx2 = np.arange(w.shape[-1]-1 ).reshape(-1, 1)

idx = idx1 + idx2

print(x[:,idx].shape)
a1 = x[:,idx]
a2 = w
print(a1)
a= np.zeros((f,o))
for i in range(f):
    for j in range(o):
        x_in = a1[j]
        print(x_in.shape)
        a[i][j] = np.sum(x_in @ w[i][j])
        print('a',a.shape)
    print(b[i].shape)
    a[i] += a[i] + b[i]
    print(a)

(2, 2, 3)
[[[1 2 3]
  [2 3 4]]

 [[2 3 4]
  [3 4 5]]]
(2, 3)
a (3, 2)
(2, 3)
a (3, 2)
()
[[31. 43.]
 [ 0.  0.]
 [ 0.  0.]]
(2, 3)
a (3, 2)
(2, 3)
a (3, 2)
()
[[31. 43.]
 [32. 44.]
 [ 0.  0.]]
(2, 3)
a (3, 2)
(2, 3)
a (3, 2)
()
[[31. 43.]
 [32. 44.]
 [33. 45.]]


In [41]:
def compute_output_size(n_in, P, F, S):
    return int(1 + (n_in + 2*P - F) / S)

In [43]:
class FC:
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)

    
    def forward(self, Z1):
        self.Z = Z1.copy()
        Z2 = np.dot(Z1, self.W) + self.B
        
        return Z2
    
    
    def backward(self, dA):
        self.dB = dA 
        self.dW = np.dot(self.Z.T, dA) 
        dZ = np.dot(dA, self.W.T) 
        
        self = self.optimizer.update(self)
        
        return dZ

In [44]:
class GetMiniBatch:
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)

    def __len__(self):
        return self._stop

    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        

    def __iter__(self):
        self._counter = 0
        return self

    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]

In [45]:
class Flat():
    def __init__(self):
        self.X_shape = None
    
    def forward(self, X):
        X_1d = X.reshape(X.shape[0], -1)
        
        self.X_shape = X.shape
        
        return X_1d

    def backward(self, X):
        X = X.reshape(self.X_shape)
        
        return X

In [46]:
class Tanh:
    def __init__(self):
        self.Z = None
    
    def forward(self, A):
        self.Z =  np.tanh(A)
        
        return  self.Z
    
    def backward(self, dZ):
        dA = dZ * (1 - self.Z**2)
        
        return dA

In [47]:
class XavierInitializer:
    def __init__(self):
        pass    
    def W(self, n_nodes1=None, n_nodes2=None, 
                  filter_num=None, C=None, filter_size=None):
        #
        if filter_num and C and filter_size is not None:
            W =  np.random.randn(filter_num, C, filter_size) / np.sqrt(filter_num)
        #
        else:
            W = np.random.randn(n_nodes1, n_nodes2) / np.sqrt(n_nodes1) 
        return W
    
    def B(self, n_nodes2):
        B = np.zeros(n_nodes2) 
        
        return B

In [48]:
class Sigmoid:
    def __init__(self):
        self.Z = None 
    def forward(self, A):
        self.Z = 1 / (1 + np.exp(-A))
        return self.Z
    def backward(self, dZ):
        dA = dZ * (1 - self.Z) * self.Z
        return dA

In [49]:
class HeInitializer:
    def __init__(self):
        pass
    def W(self, n_nodes1=None, n_nodes2=None,filter_num=None, C=None, filter_size=None):
      
        if filter_num and C and filter_size is not None:
            W = np.random.randn(filter_num, C, filter_size) * np.sqrt(2 / filter_num)
        else:
            W = np.random.randn(n_nodes1, n_nodes2) * np.sqrt(2 / n_nodes1)
        return W

    def B(self, n_nodes2):
        B = np.zeros(n_nodes2) 
        return B

In [50]:
class AdaGrad:
    def __init__(self, lr):
        self.lr = lr 
        self.H_W= None 
        self.H_B = None

    def update(self, layer):
        if self.H_W is None:
            self.H_W = np.zeros(layer.W.shape)
        if self.H_B is None:
            self.H_B = np.zeros(layer.B.shape)
        
        self.H_W += (layer.dW / layer.dB.shape[0]) ** 2 
        self.H_B += (layer.dB.mean(axis=0)) **2
        layer.W -= self.alpha / np.sqrt(self.H_W + 1e-7) * layer.dW / layer.dB.shape[0] 
        layer.B -= self.alpha / np.sqrt(self.H_B + 1e-7) * layer.dB.mean(axis=0) 
        
        return layer

In [51]:
class ReLU:
    def __init__(self):
        self.X = None
    def forward(self, A):
        self.X = A.copy()
        Z = np.maximum(0, A)
        return Z
    def backward(self, dZ):
        dA = np.where(self.X > 0, dZ, 0)
        return dA

In [52]:
class SGD:
    def __init__(self, lr):
        self.lr = lr
    def update(self, layer):
        ## layer.dB=partial derivatives of loss
        layer.W -= self.lr* layer.dW / layer.dB.shape[0] #(n_nodes1, n_nodes2)
        layer.B -= self.lr* layer.dB.mean(axis=0) #(n_nodes2)

In [53]:
class Softmax:
    def __init__(self):
        self.Z = None
    
    def forward(self, A):
        c = np.max(A)
        exp_A = np.exp(A - c)
        sum_exp_A = np.sum(exp_A, axis=1).reshape(-1, 1)

        self.Z = exp_A / sum_exp_A  
        return self.Z

    def backward(self, y):
        loss_sum = np.sum(y * np.log(self.Z), axis=1)
        loss = -np.mean(loss_sum)
        
        dA = self.Z - y    
        return dA

In [68]:
class Conv1d:
    """
    A class of a convolution 1D layer
    Parameters
    ----------
    F : int
      filter size
    ch_in : int
      input channels
    ch_out : int
      output channels
    initializer: instance of initialization method
    optimizer: instance of optimization method
    """
    def __init__(self, F, ch_in, ch_out, initializer, optimizer):
        self.F = F
        self.ch_in = ch_in
        self.ch_out = ch_out
        self.optimizer = optimizer
        self.W = np.empty((ch_out, ch_in, F))
        for ch in np.arange(0, ch_out):
            self.W[ch] = initializer.W(ch_in, F)
        self.B = initializer.B(ch_out)

    def forward(self, X):
        """
        forward propagation, finding A
        Parameters
        ----------
        X : (ch_in, number of features)
        Return
        ----------
        A : (number of output channels, n_out)
        """
        self.X = X
        self.n_out = compute_output_size(self.X.shape[1], 0, self.F, 1)
        A = np.empty((self.ch_out, self.n_out))
        for ch in np.arange(0, self.ch_out):
            for i in np.arange(0, self.n_out):
                A[ch, i] =np.dot( np.sum(X[:, i:i+self.F] ,self.W[ch]) + self.B[ch])
        return A

    def backward(self, dA):
        """
        Backward
        Parameters
        ----------
        dA : The following forms of ndarray, shape (ch_out, n_out)
            Gradient flowing from behind
        Returns
        ----------
        dZ : The following forms of ndarray, shape (ch_in, number of features)
            Gradient to flow forward
        """
        #update dW
        self.dW = np.empty((self.ch_out, self.ch_in, self.F))
        for c_out in np.arange(0, self.ch_out):
            for c_in in np.arange(0, self.ch_in):
                for s in np.arange(0, self.F):
                    self.dW[c_out, c_in, s] = np.sum(dA[c_out] * self.X[c_in, s:s+self.n_out])
        #update dB
        self.dB = np.sum(dA, axis=1)
        #update dZ
        self.dZ = np.empty((self.ch_in, self.X.shape[1]))
        for c_in in np.arange(0, self.ch_in):
            for j in np.arange(0, self.X.shape[1]):
                total = 0.0
                for c_out in np.arange(0, self.ch_out):
                    for s in np.arange(0, self.F):
                        if (j - s >= 0 and j - s < self.n_out):
                            total += dA[c_out, j - s] * self.W[c_out, c_in, s]
                self.dZ[c_in, j] = total
        # update new W, B
        self = self.optimizer.update(self)
        return self.dZ

In [54]:
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma 
        

    def W(self, n_nodes1=None, n_nodes2=None,filter_num=None, C=None, filter_size=None):

        if filter_num is not None and C is not None and filter_size is not None:
            W =  self.sigma * np.random.randn(filter_num, C, filter_size) 
        if n_nodes1 is not None and n_nodes2 is not None:
            W =  self.sigma * np.random.randn(n_nodes1, n_nodes2) 
        return W
    
    
    def B(self, n_nodes2):
        B = np.zeros(n_nodes2) 
        return B

In [69]:
class Scratchch1dCNNNeuralNetrowkClassifier():
    def __init__(self, epoc=10, activation='relu', solver='adagrad', alpha=0.005, batch_size=10, initial='he', sigma=0.01, n_nodes1=50,n_nodes2=25, filter_num=3,  filter_size=7, verbose=True):
        self.epoc= epoc  
        self.activation=activation 
        self.solver=solver 
        self.alpha=alpha
        self.batch_size=batch_size
        self.initial=initial    
        self.sigma=sigma         
        self.n_nodes1=n_nodes1  
        self.n_nodes2=n_nodes2   
        self.filter_num=filter_num   
        self.filter_size=filter_size    
        self.verbose=verbose        
        
        
        self.conv1=None
        self.FC1= None 
        self.FC2= None
        self.activation1= None 
        self.activation2= None
        self.activation3= None 
        self.loss_list= None 
        self.mini_loss_list= None 
        self.val_loss_list= None
        self.mini_loss_list= None 
        self.flat=None
    
    
    def fit(self, X, y, X_val=None, y_val=None):

        #one_hot
        n_output = np.unique(y).shape[0]
        y_onehot = self._one_hot(y, n_output)
        
        #
        train_mini_batch = GetMiniBatch(X, y_onehot, self.batch_size)
        
        #val
        if X_val is not None and y_val is not None:       
            y_val_onehot = self._one_hot(y_val, n_output) #one-hot
            test_mini_batch = GetMiniBatch(X_val, y_val_onehot) #
        
        
        #
        if self.activation == 'sigmoid':
            activate1 = Sigmoid()
            activate2 = Sigmoid()
            activate3 = Sigmoid()
        elif self.activation == 'tanh':
            activate1 = Tanh()
            activate2 = Tanh()
            activate3 = Tanh()
        elif self.activation == 'relu':
            activate1 = ReLU()
            activate2 = ReLU()
            activate3 = ReLU()

        
        #
        if self.solver == 'sgd':
            optimizer1 = SGD(self.alpha)
            optimizer2 = SGD(self.alpha)
            optimizer3 = SGD(self.alpha) 
            optimizer4 = SGD(self.alpha)
            optimizer5 = SGD(self.alpha) 
        elif self.solver == 'adagrad':
            optimizer1 = AdaGrad(self.alpha)
            optimizer2 = AdaGrad(self.alpha)
            optimizer3 = AdaGrad(self.alpha)
            optimizer4 = AdaGrad(self.alpha)
            optimizer5 = AdaGrad(self.alpha)
            
        #
        if self.initial == 'simple':
            initializer1 = SimpleInitializer(self.sigma)
            initializer2 = SimpleInitializer(self.sigma)
            initializer3 = SimpleInitializer(self.sigma)
            initializer4 = SimpleInitializer(self.sigma)
            initializer5 = SimpleInitializer(self.sigma)
        elif self.initial == 'xavier':
            initializer1 = XavierInitializer()
            initializer2 = XavierInitializer()
            initializer3 = XavierInitializer()
            initializer4 = XavierInitializer()
            initializer5 = XavierInitializer()
        elif self.initial == 'he':
            initializer1 = HeInitializer()
            initializer2 = HeInitializer()
            initializer3 = HeInitializer()
            initializer4 = HeInitializer()
            initializer5 = HeInitializer()

        #
       # self.conv1 = Conv1d(initializer1, optimizer1, self.filter_num, X.shape[1], self.filter_size)
        self.conv1 = Conv1d(self.filter_num, X.shape[1], self.filter_size,initializer1, optimizer1)

        self.activation1 = activate1
        
        #
        self.flat = Flat()
        out = self.filter_num * (X.shape[1] - (self.filter_size - 1))

        #
        self.FC1 = FC(out, self.n_nodes1, initializer2, optimizer2) 
        self.activation2 = activate2
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, initializer3, optimizer3) 
        self.activation3 = activate3
        self.FC3 = FC(self.n_nodes2, y_onehot.shape[1], initializer4, optimizer4)
        self.activation4 = Softmax()
        
        #
        self.loss_list=[]
        self.val_loss_list=[]
        
        #
        for i in range(self.epoc):
            
            #
            self.mini_loss_list = []
            
            #
            for mini_X_train, mini_y_train in train_mini_batch:
                
                #
                A1 = self.conv1.forward(mini_X_train) 
                Z1 = self.activation1.forward(A1)    
                F1 = self.flat.forward(Z1)
                
                #
                A2 = self.FC1.forward(F1)                   
                Z2 = self.activation2.forward(A2) 
                A3 = self.FC2.forward(Z2)             
                Z3 = self.activation3.forward(A3)
                A4 = self.FC3.forward(Z3)             
                Z4 = self.activation4.forward(A4)   
                
                #
                dA4, mini_loss = self.activation4.backward(mini_y_train) 
                dZ4 = self.FC3.backward(dA4)          
                dA3 = self.activation3.backward(dZ4) 
                dZ3 = self.FC2.backward(dA3)            
                dA2 = self.activation2.backward(dZ3) 
                dZ2 = self.FC1.backward(dA2)    
                
                #
                dF1 = self.flat.backward(dZ2)
                
                
                dA2 = self.activation1.backward(dF1)
                dZ1 = self.conv1.backward(dA2)

                #
                self.mini_loss_list.append(mini_loss)

            #
            loss = np.mean(self.mini_loss_list)
            self.loss_list.append(loss)

            
            #
            if X_val is not None and y_val is not None:
                
                self.mini_val_loss_list = []
                for mini_X_val, mini_y_val in test_mini_batch:
              
                    #
                    A1 = self.conv1.forward(mini_X_val)
                    Z1 = self.activation1.forward(A1)
                    F1 = self.flat.forward(Z1)
                    A2 = self.FC1.forward(F1)
                    Z2 = self.activation2.forward(A2)
                    A3 = self.FC2.forward(Z2)
                    Z3 = self.activation3.forward(A3)
                    A4 = self.FC3.forward(Z3) 
                    Z4 = self.activation4.forward(A4)

                    #
                    _, mini_val_loss = self.activation4.backward(mini_y_val)
                    self.mini_val_loss_list.append(mini_val_loss)

                val_loss = np.mean(self.mini_val_loss_list)
                self.val_loss_list.append(val_loss)

                
            #
            if self.verbose == True:
                print('Processus d\'apprentissage des données de vérification' + str(i + 1) + 'epoc : ' + str(self.loss_list[i]))
                #
            if X_val is not None or y_val is not None:
                print('Processus d\'apprentissage des données de vérification' + str(i + 1) + 'epoc: ' + str(self.val_loss_list[i]))
                    
                
    def predict(self, X):
    
        #
        A1 = self.conv1.forward(X) 
        Z1 = self.activation1.forward(A1) 
        
        #
        F1 = self.flat.forward(Z1)
        
        #
        A2 = self.FC1.forward(F1)
        Z2 = self.activation2.forward(A2)
        A3 = self.FC2.forward(Z2)
        Z3 = self.activation3.forward(A3)
        A4 = self.FC3.forward(Z3)
        y_pred = self.activation4.forward(A4)  
                       
        return np.argmax(y_pred, axis=1)
            

    def _one_hot(self, y, n_output):
        #
        one_hot = np.zeros((n_output, y.shape[0]))
        
        #
        for idx, val in enumerate(y.astype(int)):
            one_hot[val, idx] = 1

        return one_hot.T

In [70]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)


X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255
print(X_train.max()) # 1.0
print(X_train.min()) # 0.0


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
print(X_train.shape) # (48000, 784)
print(X_val.shape) # (12000, 784)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  import sys


1.0
0.0
(48000, 784)
(12000, 784)


In [ ]:
model = Scratchch1dCNNNeuralNetrowkClassifier()

model.fit(X_train,y_train,X_val,y_val)